# Lesson 1: Advanced RAG Pipeline

In [1]:
# Import OpenAI API key from prompt user input on keypass

import os
# import sys
# Get API key from user input

api_key = input("Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = api_key

# Get HuggingFace API key from user input
huggingface_key = input("Enter your HuggingFace API key: ")
os.environ["HUGGINGFACE_API_KEY"] = huggingface_key


In [2]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [3]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [4]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.schema.Document'>
Doc ID: 7b658a26-1ad2-4daf-939f-8d4a270063fa
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


## Basic RAG pipeline

In [5]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [6]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

In [7]:
query_engine = index.as_query_engine()

In [8]:
response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)
print(str(response))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


When finding projects to build your experience, there are several steps you can take. First, you can join existing projects by asking to join someone else's project if they have an idea. Additionally, you can continue reading, taking courses, and talking to domain experts to come up with new ideas. It is also helpful to focus on a specific application area where machine learning has not yet been applied. This can give you the opportunity to explore unique and creative applications that no one else has done yet. Finally, you can develop a side hustle or personal project that may or may not develop into something bigger, as this can stir your creative juices and strengthen bonds with collaborators.


## Evaluation setup using TruLens

In [9]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the keys to building a career in AI?
How can teamwork contribute to success in AI?
What is the importance of networking in AI?
What are some good habits to develop for a successful career?
How can altruism be beneficial in building a career?
What is imposter syndrome and how does it relate to AI?
Who are some accomplished individuals who have experienced imposter syndrome?
What is the first step to becoming good at AI?
What are some common challenges in AI?
Is it normal to find parts of AI challenging?


In [10]:
# You can try your own question:
new_question = "What is the right AI job for me?"
eval_questions.append(new_question)

In [11]:
print(eval_questions)

['What are the keys to building a career in AI?', 'How can teamwork contribute to success in AI?', 'What is the importance of networking in AI?', 'What are some good habits to develop for a successful career?', 'How can altruism be beneficial in building a career?', 'What is imposter syndrome and how does it relate to AI?', 'Who are some accomplished individuals who have experienced imposter syndrome?', 'What is the first step to becoming good at AI?', 'What are some common challenges in AI?', 'Is it normal to find parts of AI challenging?', 'What is the right AI job for me?']


In [12]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


For the classroom, we've written some of the code in helper functions inside a utils.py file.  
- You can view the utils.py file in the file directory by clicking on the "Jupyter" logo at the top of the notebook.
- In later lessons, you'll get to work directly with the code that's currently wrapped inside these helper functions, to give you more options to customize your RAG pipeline.

In [13]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

In [14]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

boto3,botocore is/are required for using BedrockEndpoint. You should be able to install it/them with
	pip install boto3 botocore


In [15]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [16]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_97549092250ca6853950a233b8a327f8,"""What are the keys to building a career in AI?""","""The keys to building a career in AI are learn...",-,"{""record_id"": ""record_hash_97549092250ca685395...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-12T00:34:59.991017"", ""...",2023-12-12T00:35:07.185762,1.0,0.9,1.00,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'prompt': 'What are the keys to bui...,"[{'args': {'source': 'PAGE 1Founder, DeepLearn...",7,2255,0.003501
1,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_e3c6a3d339716f8a8ab9523ebcebed62,"""How can teamwork contribute to success in AI?""","""Collaborating and working in teams is crucial...",-,"{""record_id"": ""record_hash_e3c6a3d339716f8a8ab...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-12T00:35:07.267605"", ""...",2023-12-12T00:35:10.570057,1.0,0.0,1.00,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'source': 'Hopefully the previous c...,3,1725,0.002638
2,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_68a6b1679eca9644f4cf15cabb3d7da4,"""What is the importance of networking in AI?""","""Networking is important in AI because it help...",-,"{""record_id"": ""record_hash_68a6b1679eca9644f4c...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-12T00:35:10.652831"", ""...",2023-12-12T00:35:13.621869,1.0,0.0,0.25,[{'args': {'prompt': 'What is the importance o...,[{'args': {'prompt': 'What is the importance o...,[{'args': {'source': 'Hopefully the previous c...,2,1711,0.002609
3,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_9a76c90e4cab3fc4341399bab0228b51,"""What are some good habits to develop for a su...","""Developing good habits is crucial for a succe...",-,"{""record_id"": ""record_hash_9a76c90e4cab3fc4341...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-12T00:35:13.704026"", ""...",2023-12-12T00:35:17.194075,1.0,0.6,1.00,[{'args': {'prompt': 'What are some good habit...,[{'args': {'prompt': 'What are some good habit...,[{'args': {'source': 'Hopefully the previous c...,3,1680,0.002563
4,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_c640ddd8cc9758a0021b49fc31a0a834,"""How can altruism be beneficial in building a ...","""Altruism can be beneficial in building a care...",-,"{""record_id"": ""record_hash_c640ddd8cc9758a0021...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-12T00:35:17.268232"", ""...",2023-12-12T00:35:20.132110,1.0,0.0,0.70,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'source': 'Hopefully the previous c...,2,1680,0.002563


In [17]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://10.0.0.21:8502 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Advanced RAG pipeline

### 1. Sentence Window retrieval

In [18]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [19]:
from utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [20]:
from utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [21]:
window_response = sentence_window_engine.query(
    "how do I get started on a personal project in AI?"
)
print(str(window_response))

To get started on a personal project in AI, it is important to first identify and scope the project. Consider what field or area of AI you are interested in exploring and choose a project that aligns with your career goals. Once you have chosen a project, focus on executing it with an eye towards career development. As you work on the project, aim to grow its scope, complexity, and impact over time. Building a portfolio of projects that shows skill progression can also be beneficial in showcasing your expertise in AI.


In [22]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

In [23]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

What are the keys to building a career in AI?
The keys to building a career in AI are learning foundational technical skills, working on projects, and finding a job, all of which is supported by being part of a community.
How can teamwork contribute to success in AI?
Teamwork can contribute to success in AI by allowing individuals to leverage the expertise and insights of their colleagues. When working on larger AI projects, the ability to collaborate effectively with teammates becomes crucial. By working together, team members can share their technical insights, make informed decisions about technical architecture or data collection, and ultimately improve the project. Additionally, being surrounded by colleagues who are dedicated, hardworking, and continuously learning can inspire individuals to do the same, leading to greater success in AI endeavors.
What is the importance of networking in AI?
Networking is important in AI because it allows individuals to connect with others who hav

In [24]:
tru.get_leaderboard(app_ids=[])

,Groundedness,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,,
Sentence Window Query Engine,0.720833,0.128571,1.0,2.272727,0.000849


In [25]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://10.0.0.21:8502



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### 2. Auto-merging retrieval

In [26]:
from utils import build_automerging_index

automerging_index = build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index"
)

In [27]:
from utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [28]:
auto_merging_response = automerging_query_engine.query(
    "How do I build a portfolio of AI projects?"
)
print(str(auto_merging_response))

> Merging 1 nodes into parent node.
> Parent node id: b2492a56-fa6c-4b07-b89e-1fa7ef4044ab.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

> Merging 1 nodes into parent node.
> Parent node id: bda46bfb-d12b-4e1b-8593-2c9946145af7.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

To build a portfolio of AI projects, it is important to start with simple undertakings and gradually progress to more complex ones. This progression will demonstrate your growth and development over time. Additionally, effective communication is crucial in order to explain your thinking and showcase the value of your work. Being able to articulate your ideas will help others see the potential in your projects and trust you with the necessary resources.


In [29]:
tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [30]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x425d392d0 is calling an instrumented method <function BaseQueryEngine.query at 0x147d54040>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x3d15ea860) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x425d392d0 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x158ccc550>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x3d15ea860) using this function.
A new object of type <class 'llama_index.retrievers.auto_merging_retriever.AutoMergingRetriever'> at 0x3d1502e30 is calling an instrumented method <function BaseRetriever.retrieve at 0x147d2f370>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on other object (0x2f55e6770) using this fun

> Merging 2 nodes into parent node.
> Parent node id: b0d84641-271d-4347-8fb7-17863b13dc53.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: b11589c3-f7f2-4064-8e1e-4c8a6020da7f.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x425d39210 is calling an instrumented method <function CompactAndRefine.get_response at 0x151355630>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x2f55e6d40) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x425d39210 is calling an instrumented method <function Refine.get_response at 0x151f12560>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x2f55e6d40) using this function.
A new object of type <class 'llama_index.llm_predictor.base.LLMPredictor'> at 0x3e3273b00 is calling an instrumented method <function LLMPredictor.predict at 0x117a30e50>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer.service_context.llm

What are the keys to building a career in AI?
The keys to building a career in AI include learning foundational technical skills, working on projects, finding a job, and being part of a community. Additionally, collaborating with others and being able to influence and be influenced by others is critical for success in AI.


A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x425d39210 is calling an instrumented method <function Refine.get_response at 0x151f12560>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x2f55e6d40) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x425d392d0 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x158ccc550>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x3d15ea860) using this function.


...

What are some common challenges in AI?
Some common challenges in AI include the highly iterative nature of AI projects and the difficulty in coming up with a plan for building a system when the time required to achieve the target accuracy is unknown. Additionally, technical challenges are also common in AI, and even experienced researchers and practitioners have struggled with them at some point.
> Merging 3 nodes into parent node.
> Parent node id: fc74e7d3-7134-4524-a20f-c0ea2555e334.
> Parent node text: PAGE 38Before we dive into the final chapter of this book, I’d like to address the serious matter...

> Merging 1 nodes into parent node.
> Parent node id: c447606d-2651-4368-9c22-cf212184be47.
> Parent node text: PAGE 38Before we dive into the final chapter of this book, I’d like to address the serious matter...



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x425d39210 is calling an instrumented method <function Refine.get_response at 0x151f12560>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x2f55e6d40) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x425d392d0 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x158ccc550>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x3d15ea860) using this function.


Is it normal to find parts of AI challenging?
Yes, it is normal to find parts of AI challenging. The context information suggests that even accomplished individuals in the AI community have faced technical challenges and struggled with certain aspects of AI. The author encourages newcomers to not be discouraged by these challenges and assures them that everyone has been in a similar position at some point.
> Merging 1 nodes into parent node.
> Parent node id: 24088e64-61fb-4575-88e4-1c4e648c4457.
> Parent node text: PAGE 31Finding the Right 
AI Job for YouCHAPTER 9
JOBS

> Merging 1 nodes into parent node.
> Parent node id: 048830de-6dde-4f55-8a3f-a67075495b3b.
> Parent node text: If you’re leaving 
a job, exit gracefully. Give your employer ample notice, give your full effort...

> Merging 1 nodes into parent node.
> Parent node id: 1a9f3d38-6dd1-4609-a627-40e604af0546.
> Parent node text: PAGE 28Using Informational 
Interviews to Find 
the Right JobCHAPTER 8
JOBS

> Merging 1 nodes i

A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x425d39210 is calling an instrumented method <function Refine.get_response at 0x151f12560>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x2f55e6d40) using this function.


What is the right AI job for me?
The right AI job for you would depend on your specific skills, interests, and goals. It is important to consider your background and experience in order to determine which role or industry would be the best fit for you. Switching either roles or industries in the field of AI can be a good starting point, and it may be easier to transition into one or the other rather than both simultaneously. It is recommended to build a portfolio of projects that demonstrate your skill progression and to conduct informational interviews to explore different AI job opportunities. Ultimately, the right AI job for you would be one that aligns with your interests, allows for skill development, and provides opportunities for growth in the field.


In [31]:
tru.get_leaderboard(app_ids=[])

,Groundedness,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,,
Automerging Query Engine,0.9,0.328571,0.988889,2.363636,0.000898


In [32]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://10.0.0.21:8502



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>